<a href="https://colab.research.google.com/github/RKDash7/R-K-Dash/blob/main/Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import hashlib
import time
import json
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import hashlib


# Blockchain class
class Blockchain:
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        # Create the first block (genesis block)
        self.create_block(previous_hash='1', proof=100)

    def create_block(self, proof, previous_hash=None):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time.time(),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1]),
        }
        self.current_transactions = []  # Reset current transactions
        self.chain.append(block)
        return block

    def add_transaction(self, sender, recipient, amount, signature,message):
        """ Adds a transaction to the current list of transactions. """
        if not self.verify_signature(sender, signature,message):
            raise Exception("Invalid signature")

        self.current_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
            'signature': signature
        })
        return self.last_block['index'] + 1

    def proof_of_work(self, last_proof):
        """ Proof of Work algorithm to find a valid nonce. """
        proof = 0
        while not self.valid_proof(last_proof, proof):
            proof += 1
        return proof

    def valid_proof(self, last_proof, proof):
        """ Validate the Proof of Work by checking if the hash starts with '0000'. """
        guess = f'{last_proof}{proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000"

    def hash(self, block):
        """ Hash a block with SHA-256. """
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def verify_signature(self, sender, signature,message):
        """ Verify the signature of a transaction. """
        public_key = sender["public_key"]
        #message = f"{sender['id']} sent to {signature['recipient']} {signature['amount']} packets"
        try:
            public_key.verify(
                signature['signature'],
                message.encode(),
                padding.PKCS1v15(),
                hashes.SHA256()
            )
            return True
        except:
            return False

    @property
    def last_block(self):
        return self.chain[-1]

# Digital Signature and RSA Key Generation
def generate_keys():
    """ Generate RSA public and private keys for signing and verification. """
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()

    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    )

    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    return private_key, public_key, private_pem, public_pem

def sign_transaction(private_key, message):
    """ Sign a message (transaction) using the sender's private key. """
    signature = private_key.sign(
        message.encode(),
        padding.PKCS1v15(),
        hashes.SHA256()
    )
    return signature
def encrypt_message(message, public_key):
      rsa_key = RSA.import_key(public_key)
      cipher = PKCS1_OAEP.new(rsa_key)
      encrypted_message = cipher.encrypt(message.encode())
      return encrypted_message
def generate_key():
      key = RSA.generate(2048)
      private_key = key.export_key()
      public_key = key.publickey().export_key()
      return private_key, public_key